Topic - Peer Group Prediction

Details about the data:
Human face image files are available, all the images are split based on the ages. 6 age groups are available, it varies from 20 to 49. In each age group multiple image files are available.


In [83]:
# importing the libraries
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import classification_report,r2_score
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
from pathlib import Path
from keras.models import load_model
from sklearn.metrics import classification_report
from tensorflow.keras.models import load_model

Read testing images and preprocess the data and split the features and the targets

In [84]:
def preprocess_test_data():
  # set the path of the folder where the images stored
    data_dir = "/content/drive/MyDrive/AGE DATASET/final_test_data/"
    # set the list of age categories available in the folder
    categories = ["2024","2529","3034","3539","4044","4549"]
    # set an image size
    IMG_SIZE = 50
    
    testing_data = []
    # function to read all the images and extract the testing images and corresponding classes
    def generate_testing_data():
        # iterate through all the age folder and through each image
        for item_ in categories:
          # create location to individual images based on the ages  
            path = os.path.join(data_dir,item_)  
            # get the classification  based each age group
            class_num = categories.index(item_)  
            # iterate over each image per age folder
            for img in (os.listdir(path)): 
                try:
                  # reading and coverting the images
                    img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
                    # resize to normalize data size
                    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  
                    # add this to our training_data
                    testing_data.append([new_array, class_num])  
                #to avoid unexpected error
                except Exception as e:  
                    pass

    generate_testing_data()
    print(len(testing_data))
        
    feature = []
    target = []
    # spli the set of features and targets
    for features,label in testing_data:
        feature.append(features)
        target.append(label)
    # convert the features and targets into numpy array format
    feature = np.array(feature).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
    target = np.array(target)
    # normalizing the test features
    X_test = feature/255
    
    # Print the shape of x_test again.
    print(X_test.shape)
    #assigning the target values into another variable
    y_test = target
    # Print the shape of y_test again
    print(y_test.shape)

    # Your code to print shape of y_test
    print(y_test.shape)    
    # Print shape of y_test [0]
    print(y_test[0].shape)
    
    # Your code to use to_categorical to convert integers to numbers. Assign the new array to the variable y_test
    Y_test = to_categorical(y_test)
    # saving the actual test data for further reference
    np.savetxt("5batch_test_target.csv", Y_test, delimiter=",")

        
    # Print shape of the array y_cat_train
    print(Y_test.shape)
    # returning the set of testing features and targets
    return X_test, Y_test


Testing model

In [85]:
def test_model(X_test, Y_test):
   # load the saved model
    CNN_model = load_model('/content/5batch_age_model_v2_.h5')
    print('Testing Started \n\n\n')

    # CNN_model.evaluate(X_test, Y_test, verbose=0)
    predictions_prob = CNN_model.predict(X_test)
    predictions = predictions_prob.argmax(axis=-1)
       
    # print('Exact prediction',accuracy_cnt)
    score = CNN_model.evaluate(X_test, Y_test, verbose=0)
    # print("Test Score: ", score[0])
    print('Test loss:', score[0])
    print("Accuracy: ", score[1]*100)

    Y_test = np.argmax(Y_test, axis=-1)
    predictions_prob = np.argmax(predictions_prob, axis=-1)
    print('\n\n')

    np.savetxt("5batch_test_target.csv", Y_test, delimiter=",")
    np.savetxt("5batch_predicted_target.csv", predictions_prob, delimiter=",")

    print('******classification report******')
    print(classification_report(Y_test, predictions_prob))
    print('__________________________')
    # displays the confusion matrix
    print('\nConfusion Matrix')
    print('__________________________')
    y_actual = pd.Series(Y_test, name='Actual')
    y_predicted = pd.Series(predictions_prob, name='Predicted')
    plot_confusion_matrix = pd.crosstab(y_actual, y_predicted)
    print(plot_confusion_matrix)
    
    # alternate function to plot the confusion matrix
    # print(confusion_matrix(Y_test, predictions_prob))

    return predictions_prob, predictions


In [86]:
if __name__ == '__main__':
    X_test, Y_test = preprocess_test_data()
    results,pred_ = test_model(X_test, Y_test)


1800
(1800, 50, 50, 1)
(1800,)
(1800,)
()
(1800, 6)
Testing Started 



57/57 [==============================] - 1s 18ms/step
Test loss: 1.927790880203247
Accuracy:  72.83333539962769



******classification report******
              precision    recall  f1-score   support

           0       0.68      0.78      0.72       300
           1       0.76      0.71      0.74       300
           2       0.72      0.76      0.74       300
           3       0.73      0.66      0.69       300
           4       0.80      0.69      0.74       300
           5       0.70      0.77      0.73       300

    accuracy                           0.73      1800
   macro avg       0.73      0.73      0.73      1800
weighted avg       0.73      0.73      0.73      1800

__________________________

Confusion Matrix
__________________________
Predicted    0    1    2    3    4    5
Actual                                 
0          233   15   18   12    8   14
1           28  214   16   13   10   19
2   